In [2]:
from utils import *
import mrf
import llm
from importlib import reload
from evaluate import *
import utils
import evaluate
reload(evaluate)
import json
import os

In [28]:
# list all files in all subdirectories
def list_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                yield os.path.join(root, file)

k = [i for i in list_files('./Experiments/trial/bin0')]

In [29]:
k

['./Experiments/trial/bin0/Rochester/Rochester_q.json',
 './Experiments/trial/bin0/Newark/Newark_q.json',
 './Experiments/trial/bin0/Columbus/Columbus_q.json',
 './Experiments/trial/bin0/BrowardCounty/BrowardCounty_q.json',
 './Experiments/trial/bin0/SanDiego/SanDiego_q.json',
 './Experiments/trial/bin0/Cambridge/Cambridge_q.json',
 './Experiments/trial/bin0/SantaClaraCounty/SantaClaraCounty_q.json',
 './Experiments/trial/bin0/ClarkCounty/ClarkCounty_q.json',
 './Experiments/trial/bin0/Dallas/Dallas_q.json',
 './Experiments/trial/bin0/LosAngeles/LosAngeles_q.json',
 './Experiments/trial/bin0/Salem/Salem_q.json',
 './Experiments/trial/bin0/SanMateoCounty/SanMateoCounty_q.json',
 './Experiments/trial/bin0/Denver/Denver_q.json',
 './Experiments/trial/bin0/JerseyCity/JerseyCity_q.json',
 './Experiments/trial/bin0/FortWorth/FortWorth_q.json',
 './Experiments/trial/bin0/Oakland/Oakland_q.json',
 './Experiments/trial/bin0/Nashville/Nashville_q.json']

In [30]:
b = [k[3], k[4]]
b

['./Experiments/trial/bin0/BrowardCounty/BrowardCounty_q.json',
 './Experiments/trial/bin0/SanDiego/SanDiego_q.json']

In [42]:
def parse_results(path):
    js = json.load(open(path))
    ground = js['ground']
    cot = []
    zero = []
    mrf = []
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        cot.append(js[key]['cot']['probability'])
        zero.append(js[key]['zero']['probability'])
        mrf.append(js[key]['mrf']['probability'])
    return ground, cot, zero, mrf
        
def metrics(ground, res):
    print(evaluate.kl_divergence(ground, res), evaluate.kl_divergence(res, ground))
    print(evaluate.total_variation_distance(ground, res))

#for j in [i for i in list_files('./Experiments/trial/bin0')]:
for j in b:
    try:
        ground, cot, zero, mrf = parse_results(j)
        cotcot = [np.array(i) for i in cot]
        zero = [np.array(i) for i in zero]
        mrf = [np.array(i) for i in mrf]
        # average
        #cotcot = np.mean(cotcot, axis=0)
        #zero = np.mean(zero, axis=0)
        #mrf = np.mean(mrf, axis=0)
        for i in range(len(cotcot)):
            print('COT')
            metrics(ground, cotcot[i])
            print('MRF')
            metrics(ground, mrf[i])
            print('ZERO')
            metrics(ground, zero[i])
        print("------------------------")
    except:
        print(j)
        print(cot)
        continue
    

COT
0.19961530390984322 0.2165540171314333
0.28216286083584663
MRF
1.0778233718550914 0.8583126927089622
0.5231091101271474
ZERO
0.2767087670820899 0.27432803871748396
0.3321628608358466
COT
0.08581586233822557 0.09716951020311979
0.1794207238259371
MRF
0.13344242938463183 0.15811287921009892
0.20978249716783123
ZERO
0.2767087670820899 0.27432803871748396
0.3321628608358466
------------------------
COT
0.6256268144021081 0.3695442238362798
0.34363429793490613
MRF
0.28635088384307145 0.3588016816783174
0.34464228684416554
ZERO
0.03590392350574148 0.03897543525772716
0.0870179776782731
COT
./Experiments/trial/bin0/SanDiego/SanDiego_q.json
[[0.2, 0.4, 0.3, 0.1, 0.0], [0.35, 0.35]]


In [124]:
k = [i for i in list_files('./Experiments/trial/bin0')]
b = ['./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json']

In [99]:
pp = '[{"Target": [{"Name": "PriceRange", "Value": ["$0-$50"]}], "Condition": [], "Probability": [0.05]}, {"Target": [{"Name": "PriceRange", "Value": ["$51-$100"]}], "Condition": [], "Probability": [0.15]}, {"Target": [{"Name": "PriceRange", "Value": ["$101-$200"]}], "Condition": [], "Probability": [0.35]}, {"Target": [{"Name": "PriceRange", "Value": ["$201-$500"]}], "Condition": [], "Probability": [0.35]}, {"Target": [{"Name": "PriceRange", "Value": ["$501+"]}], "Condition": [], "Probability": [0.1]}, {"Target": [{"Name": "Location", "Value": ["Downtown"]}], "Condition": [], "Probability": [0.25]}, {"Target": [{"Name": "Location", "Value": ["Beachfront"]}], "Condition": [], "Probability": [0.15]}, {"Target": [{"Name": "Location", "Value": ["Suburban"]}], "Condition": [], "Probability": [0.3]}, {"Target": [{"Name": "Location", "Value": ["NearAttractions"]}], "Condition": [], "Probability": [0.2]}, {"Target": [{"Name": "Location", "Value": ["Remote"]}], "Condition": [], "Probability": [0.1]}, {"Target": [{"Name": "PropertyType", "Value": ["Apartment"]}], "Condition": [], "Probability": [0.4]}, {"Target": [{"Name": "PropertyType", "Value": ["House"]}], "Condition": [], "Probability": [0.3]}, {"Target": [{"Name": "PropertyType", "Value": ["Studio"]}], "Condition": [], "Probability": [0.15]}, {"Target": [{"Name": "PropertyType", "Value": ["SharedRoom"]}], "Condition": [], "Probability": [0.1]}, {"Target": [{"Name": "PropertyType", "Value": ["LuxuryProperty"]}], "Condition": [], "Probability": [0.05]}, {"Target": [{"Name": "Bedrooms", "Value": ["Studio"]}], "Condition": [], "Probability": [0.2]}, {"Target": [{"Name": "Bedrooms", "Value": ["1"]}], "Condition": [], "Probability": [0.3]}, {"Target": [{"Name": "Bedrooms", "Value": ["2"]}], "Condition": [], "Probability": [0.25]}, {"Target": [{"Name": "Bedrooms", "Value": ["3"]}], "Condition": [], "Probability": [0.15]}, {"Target": [{"Name": "Bedrooms", "Value": ["4+"]}], "Condition": [], "Probability": [0.1]}, {"Target": [{"Name": "Amenities", "Value": ["Basic"]}], "Condition": [], "Probability": [0.3]}, {"Target": [{"Name": "Amenities", "Value": ["Standard"]}], "Condition": [], "Probability": [0.4]}, {"Target": [{"Name": "Amenities", "Value": ["Premium"]}], "Condition": [], "Probability": [0.3]}, {"Target": [{"Name": "Seasonality", "Value": ["LowSeason"]}], "Condition": [], "Probability": [0.4]}, {"Target": [{"Name": "Seasonality", "Value": ["HighSeason"]}], "Condition": [], "Probability": [0.6]}, {"Target": [{"Name": "Rating", "Value": ["1-2stars"]}], "Condition": [], "Probability": [0.1]}, {"Target": [{"Name": "Rating", "Value": ["3-4stars"]}], "Condition": [], "Probability": [0.5]}, {"Target": [{"Name": "Rating", "Value": ["4.5-5stars"]}], "Condition": [], "Probability": [0.4]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Location", "Value": ["Downtown"]}], "BatchProbability": [[0.05], [0.15], [0.4], [0.3], [0.1]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Location", "Value": ["Beachfront"]}], "BatchProbability": [[0.02], [0.1], [0.25], [0.4], [0.23]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Location", "Value": ["Suburban"]}], "BatchProbability": [[0.1], [0.3], [0.4], [0.15], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Location", "Value": ["NearAttractions"]}], "BatchProbability": [[0.05], [0.2], [0.35], [0.3], [0.1]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Location", "Value": ["Remote"]}], "BatchProbability": [[0.15], [0.4], [0.3], [0.1], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "PropertyType", "Value": ["Apartment"]}], "BatchProbability": [[0.1], [0.3], [0.4], [0.15], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "PropertyType", "Value": ["House"]}], "BatchProbability": [[0.05], [0.15], [0.3], [0.4], [0.1]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "PropertyType", "Value": ["Studio"]}], "BatchProbability": [[0.2], [0.5], [0.25], [0.05], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "PropertyType", "Value": ["SharedRoom"]}], "BatchProbability": [[0.4], [0.5], [0.1], [0], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "PropertyType", "Value": ["LuxuryProperty"]}], "BatchProbability": [[0], [0], [0.1], [0.4], [0.5]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Bedrooms", "Value": ["Studio"]}], "BatchProbability": [[0.2], [0.5], [0.25], [0.05], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Bedrooms", "Value": ["1"]}], "BatchProbability": [[0.1], [0.4], [0.4], [0.1], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Bedrooms", "Value": ["2"]}], "BatchProbability": [[0.05], [0.2], [0.5], [0.2], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Bedrooms", "Value": ["3"]}], "BatchProbability": [[0.02], [0.1], [0.4], [0.4], [0.08]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Bedrooms", "Value": ["4+"]}], "BatchProbability": [[0], [0.05], [0.2], [0.5], [0.25]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Amenities", "Value": ["Basic"]}], "BatchProbability": [[0.1], [0.4], [0.4], [0.1], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Amenities", "Value": ["Standard"]}], "BatchProbability": [[0.05], [0.25], [0.4], [0.25], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Amenities", "Value": ["Premium"]}], "BatchProbability": [[0], [0.05], [0.2], [0.5], [0.25]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Seasonality", "Value": ["LowSeason"]}], "BatchProbability": [[0.15], [0.35], [0.35], [0.1], [0.05]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Seasonality", "Value": ["HighSeason"]}], "BatchProbability": [[0.05], [0.2], [0.4], [0.25], [0.1]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Rating", "Value": ["1-2stars"]}], "BatchProbability": [[0.3], [0.5], [0.15], [0.05], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Rating", "Value": ["3-4stars"]}], "BatchProbability": [[0.1], [0.4], [0.4], [0.1], [0]]}, {"BatchTarget": [{"Name": "PriceRange", "Value": ["$0-$50", "$51-$100", "$101-$200", "$201-$500", "$501+"]}], "Condition": [{"Name": "Rating", "Value": ["4.5-5stars"]}], "BatchProbability": [[0.05], [0.15], [0.35], [0.35], [0.1]]}, {"BatchTarget": [{"Name": "Bedrooms", "Value": ["Studio", "1", "2", "3", "4+"]}], "Condition": [{"Name": "PropertyType", "Value": ["House"]}], "BatchProbability": [[0.05], [0.1], [0.25], [0.3], [0.3]]}, {"BatchTarget": [{"Name": "Bedrooms", "Value": ["Studio", "1", "2", "3", "4+"]}], "Condition": [{"Name": "PropertyType", "Value": ["Apartment"]}], "BatchProbability": [[0.2], [0.5], [0.25], [0.05], [0.0]]}, {"BatchTarget": [{"Name": "Bedrooms", "Value": ["Studio"]}], "Condition": [{"Name": "PropertyType", "Value": ["Studio"]}], "BatchProbability": [[1.0]]}, {"BatchTarget": [{"Name": "Bedrooms", "Value": ["Studio", "1"]}], "Condition": [{"Name": "PropertyType", "Value": ["SharedRoom"]}], "BatchProbability": [[0.9], [0.1]]}, {"BatchTarget": [{"Name": "Bedrooms", "Value": ["Studio", "1", "2", "3", "4+"]}], "Condition": [{"Name": "PropertyType", "Value": ["LuxuryProperty"]}], "BatchProbability": [[0.05], [0.1], [0.2], [0.3], [0.35]]}, {"BatchTarget": [{"Name": "PropertyType", "Value": ["Apartment", "House", "Studio", "SharedRoom", "LuxuryProperty"]}], "Condition": [{"Name": "Location", "Value": ["Beachfront"]}], "BatchProbability": [[0.4], [0.3], [0.1], [0.1], [0.1]]}, {"BatchTarget": [{"Name": "PropertyType", "Value": ["Apartment", "House", "Studio", "SharedRoom", "LuxuryProperty"]}], "Condition": [{"Name": "Location", "Value": ["Suburban"]}], "BatchProbability": [[0.3], [0.5], [0.1], [0.05], [0.05]]}, {"BatchTarget": [{"Name": "PropertyType", "Value": ["Apartment", "House", "Studio", "SharedRoom", "LuxuryProperty"]}], "Condition": [{"Name": "Location", "Value": ["NearAttractions"]}], "BatchProbability": [[0.5], [0.2], [0.2], [0.05], [0.05]]}, {"BatchTarget": [{"Name": "PropertyType", "Value": ["Apartment", "House", "Studio", "SharedRoom", "LuxuryProperty"]}], "Condition": [{"Name": "Location", "Value": ["Remote"]}], "BatchProbability": [[0.2], [0.5], [0.1], [0.1], [0.1]]}, {"BatchTarget": [{"Name": "Seasonality", "Value": ["HighSeason", "LowSeason"]}], "Condition": [{"Name": "Location", "Value": ["Downtown"]}], "BatchProbability": [[0.6], [0.4]]}, {"BatchTarget": [{"Name": "Seasonality", "Value": ["HighSeason", "LowSeason"]}], "Condition": [{"Name": "Location", "Value": ["Beachfront"]}], "BatchProbability": [[0.8], [0.2]]}, {"BatchTarget": [{"Name": "Seasonality", "Value": ["HighSeason", "LowSeason"]}], "Condition": [{"Name": "Location", "Value": ["Suburban"]}], "BatchProbability": [[0.5], [0.5]]}, {"BatchTarget": [{"Name": "Seasonality", "Value": ["HighSeason", "LowSeason"]}], "Condition": [{"Name": "Location", "Value": ["NearAttractions"]}], "BatchProbability": [[0.7], [0.3]]}, {"BatchTarget": [{"Name": "Seasonality", "Value": ["HighSeason", "LowSeason"]}], "Condition": [{"Name": "Location", "Value": ["Remote"]}], "BatchProbability": [[0.4], [0.6]]}, {"BatchTarget": [{"Name": "Rating", "Value": ["1-2stars", "3-4stars", "4.5-5stars"]}], "Condition": [{"Name": "PropertyType", "Value": ["Apartment"]}], "BatchProbability": [[0.2], [0.5], [0.3]]}, {"BatchTarget": [{"Name": "Rating", "Value": ["1-2stars", "3-4stars", "4.5-5stars"]}], "Condition": [{"Name": "PropertyType", "Value": ["House"]}], "BatchProbability": [[0.1], [0.6], [0.3]]}, {"BatchTarget": [{"Name": "Rating", "Value": ["1-2stars", "3-4stars", "4.5-5stars"]}], "Condition": [{"Name": "PropertyType", "Value": ["Studio"]}], "BatchProbability": [[0.3], [0.5], [0.2]]}, {"BatchTarget": [{"Name": "Rating", "Value": ["1-2stars", "3-4stars", "4.5-5stars"]}], "Condition": [{"Name": "PropertyType", "Value": ["SharedRoom"]}], "BatchProbability": [[0.4], [0.5], [0.1]]}, {"BatchTarget": [{"Name": "Rating", "Value": ["1-2stars", "3-4stars", "4.5-5stars"]}], "Condition": [{"Name": "PropertyType", "Value": ["LuxuryProperty"]}], "BatchProbability": [[0.05], [0.3], [0.65]]}]'

In [100]:
pp = json.loads(pp)

In [101]:
pp

[{'Target': [{'Name': 'PriceRange', 'Value': ['$0-$50']}],
  'Condition': [],
  'Probability': [0.05]},
 {'Target': [{'Name': 'PriceRange', 'Value': ['$51-$100']}],
  'Condition': [],
  'Probability': [0.15]},
 {'Target': [{'Name': 'PriceRange', 'Value': ['$101-$200']}],
  'Condition': [],
  'Probability': [0.35]},
 {'Target': [{'Name': 'PriceRange', 'Value': ['$201-$500']}],
  'Condition': [],
  'Probability': [0.35]},
 {'Target': [{'Name': 'PriceRange', 'Value': ['$501+']}],
  'Condition': [],
  'Probability': [0.1]},
 {'Target': [{'Name': 'Location', 'Value': ['Downtown']}],
  'Condition': [],
  'Probability': [0.25]},
 {'Target': [{'Name': 'Location', 'Value': ['Beachfront']}],
  'Condition': [],
  'Probability': [0.15]},
 {'Target': [{'Name': 'Location', 'Value': ['Suburban']}],
  'Condition': [],
  'Probability': [0.3]},
 {'Target': [{'Name': 'Location', 'Value': ['NearAttractions']}],
  'Condition': [],
  'Probability': [0.2]},
 {'Target': [{'Name': 'Location', 'Value': ['Remote

In [107]:
type(pp)

list

In [130]:
import mrf
import importlib
importlib.reload(mrf)

k = [i for i in list_files('./Experiments/trial/bin0')]


for j in b[:1]:
    print(j)
    js = json.load(open(j))
    for key in js.keys():
        if key in ['question', 'ground']:
            continue
        #print(key)
        query = js[key]['mrf']['query']
        query = json.loads(query)
        query['Queries'][0]['Condition'] = [query['Queries'][0]['Condition'][0]]
        print(query)
        test = mrf.Brute(verbose=True)
        test.from_json(vars=js[key]['mrf']['json']['Variables'], constraints=pp)
        #test.from_json(pp)
        test.update(10000)
        test.query(query=query)
        break


./Experiments/trial/bin0/LosAngeles/LosAngeles_q.json
{'Queries': [{'BatchTarget': [{'Name': 'PriceRange', 'Value': ['$0-$50', '$51-$100', '$101-$200', '$201-$500', '$501+']}], 'Condition': [{'Name': 'Bedrooms', 'Value': ['2', '3', '4+']}]}]}

Variable:  PriceRange
Value:  $0-$50, $51-$100, $101-$200, $201-$500, $501+ 


Variable:  Location
Value:  Downtown, Beachfront, Suburban, NearAttractions, Remote 


Variable:  PropertyType
Value:  Apartment, House, Studio, SharedRoom, LuxuryProperty 


Variable:  Bedrooms
Value:  Studio, 1, 2, 3, 4+ 


Variable:  Amenities
Value:  Basic, Standard, Premium 


Variable:  Seasonality
Value:  LowSeason, HighSeason 


Variable:  Rating
Value:  1-2stars, 3-4stars, 4.5-5stars 

-----------------------------------------------
Finished building
Features:  206
Constraints:  206
-----------------------------------------------
Loss:  tensor(20.0290, grad_fn=<SumBackward0>) tensor(0.9826, grad_fn=<MaxBackward1>)
torch.return_types.topk(
values=tensor([0.9826

In [127]:
qqq = query.copy()
qqq['Queries'][0]['Condition'] = []
qqq
test.query(query=qqq)

[0.08379947394132614,
 0.26133954524993896,
 0.35113126039505005,
 0.2245233952999115,
 0.07920629531145096]

In [132]:
test.query(query=query)


[0.026790698990225792,
 0.10898080468177795,
 0.380437433719635,
 0.35317015647888184,
 0.13062085211277008]

In [111]:
js[key]['mrf']['json']

{'Variables': [{'Name': 'PriceRange',
   'Value': ['$0-$50', '$51-$100', '$101-$200', '$201-$500', '$501+']},
  {'Name': 'Location',
   'Value': ['Downtown',
    'Beachfront',
    'Suburban',
    'NearAttractions',
    'Remote']},
  {'Name': 'PropertyType',
   'Value': ['Apartment', 'House', 'Studio', 'SharedRoom', 'LuxuryProperty']},
  {'Name': 'Bedrooms', 'Value': ['Studio', '1', '2', '3', '4+']},
  {'Name': 'Amenities', 'Value': ['Basic', 'Standard', 'Premium']},
  {'Name': 'Seasonality', 'Value': ['LowSeason', 'HighSeason']},
  {'Name': 'Rating', 'Value': ['1-2stars', '3-4stars', '4.5-5stars']}],
 'Constraints': [{'Target': [{'Name': 'PriceRange', 'Value': ['$0-$50']}],
   'Condition': [],
   'Probability': [0.05]},
  {'Target': [{'Name': 'PriceRange', 'Value': ['$51-$100']}],
   'Condition': [],
   'Probability': [0.15]},
  {'Target': [{'Name': 'PriceRange', 'Value': ['$101-$200']}],
   'Condition': [],
   'Probability': [0.35]},
  {'Target': [{'Name': 'PriceRange', 'Value': ['$20

In [133]:
a = test.query(query)
a

[0.026790698990225792,
 0.10898080468177795,
 0.380437433719635,
 0.35317015647888184,
 0.13062085211277008]

In [115]:
# print rounded to 3 decimal places
print(np.round(a, 2))

[0.03 0.11 0.38 0.36 0.13]


In [134]:
b = [0.01,0.02,0.24,0.48,0.26]
evaluate.total_variation_distance(a, b)

0.2512089643999934

In [135]:
query

{'Queries': [{'BatchTarget': [{'Name': 'PriceRange',
     'Value': ['$0-$50', '$51-$100', '$101-$200', '$201-$500', '$501+']}],
   'Condition': [{'Name': 'Bedrooms', 'Value': ['2', '3', '4+']}]}]}